# 构造用于三维卷积神经网络的数据集
- 用周围19个像素的值取预测一个值，貌似太大了
- 将得到的数据集储存起来，让卷积模块调用

In [1]:
import numpy as np
import tensorflow as tf
import scipy.io as sio
from sklearn.model_selection import train_test_split

In [2]:
data_path = os.path.join('E:\Eric_HSI\hyperspectral_datasets')
data = sio.loadmat(os.path.join(data_path, 'Indian_pines_corrected.mat'))
labels = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))

window_size=19
input_size=200

In [3]:
# 数据
# print(mat.keys())
features = data['data']
features_shape = features.shape
print(features.shape)
# 标签
labels = labels['groundT']
labels = np.reshape(labels, (-1, 1))   # 把标签变为2维， (21025, 1)
labels.shape

(145, 145, 200)


(21025, 1)

In [4]:
# 创建一个 (21025, 19, 19, 200) 的数组装数据
dataset = np.zeros((labels.shape[0], window_size, window_size, input_size), dtype='float32')

# parameter？？
parameter_b = np.array([i - window_size // 2 for i in range(window_size)])

In [5]:
for i in range(features_shape[0]):
    for j in range(features_shape[1]):
        index = i * features_shape[1] + j
        for p in range(window_size):
            for q in range(window_size):
                dataset[index][p][q] = \
                    features[(i + parameter_b[p]) % features_shape[0]][(j + parameter_b[q]) % features_shape[1]]

In [6]:
# 删除所有没有标签的像素
print(labels.shape)
index = np.where(np.reshape(labels, (-1)) == 0)   # 返回坐标tuple(ndarray)
labels = np.delete(labels, index, axis=0)
print(labels.shape)

(21025, 1)
(10249, 1)


# 中间都在做一些删除拆分的操作

In [7]:
tmp = np.unique(labels)   # array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])
for i in range(len(labels)):
    labels[i][0] = np.where(tmp == labels[i][0])[0][0]  # (array([1], dtype=int64),) 第一个[0]索引出元祖中的ndarray，第二[0]索引出ndarray中的int值,处理后，所有的标签值-1 ，因为本来中标签没用0，但是经过Len(labels)，np.where 定位索引位置，又从0开始，所以标签值-1

In [8]:
dataset = np.delete(dataset, index, axis=0)
labels = np.reshape(labels, (-1))  # 降维

In [9]:
dataset = dataset.swapaxes(1, 3)      # 交换轴
print('dataset_shape: ', dataset.shape)

# input : 输入，其格式为[batch, in_depth, in_height, in_width, in_channels].可以这样理解，输入batch个 长*宽*高 的图像，添加一个轴channels为1
dataset = dataset[:, :, :, :, np.newaxis]     # 添加轴channels
print('dataset_shape: ', dataset.shape)

# before dataset_shape:  (10249, 200, 19, 19)
# after dataset_shape:  (10249, 200, 19, 19, 1)
# print(Y.shape)  # (10249,)

dataset_shape:  (10249, 200, 19, 19)
dataset_shape:  (10249, 200, 19, 19, 1)


In [10]:
# 转化为类别变量
classes = 16
labels = tf.keras.utils.to_categorical(labels, classes)

In [11]:
# 标准化
dataset = dataset.astype('float32')
dataset -= np.mean(dataset)
dataset /= np.max(dataset)

In [12]:
# Split the data
x_train_new, y_train_new, x_val_new, y_val_new = train_test_split(dataset, labels, test_size=0.8, random_state=4)

In [13]:
x_train_new.shape, y_train_new.shape, x_val_new.shape, y_val_new.shape

((2049, 200, 19, 19, 1), (8200, 200, 19, 19, 1), (2049, 16), (8200, 16))

In [14]:
path_x_train = r"E:\Eric_HSI\excise2\np_data\x_train" + ".npy"
path_x_val = r"E:\Eric_HSI\excise2\np_data\x_val" + ".npy"
path_y_train = r"E:\Eric_HSI\excise2\np_data\y_train" + ".npy"
path_y_val = r"E:\Eric_HSI\excise2\np_data\y_val" + ".npy"


np.save(path_x_train, x_train_new)
np.save(path_x_val, x_val_new)
np.save(path_y_train, y_train_new)
np.save(path_y_val, y_val_new)